# Create LDA for Corpus<a id='top'></a>

0. Download an available corpus or create a new one. For the latter, create a JSON file for each subcorpus/group of texts of your corpus; each text is then a line in a JSON file. One way to do this is to crawl websites using [scrapy](https://scrapy.org) with these flags: "-o result.json -t json" (see [sample crawlers](./scripts/scraper/spiders) and [example item file](./scripts/scraper/items.py)). An example JSON file is [here](./scripts/example.json).
1. [Prepare corpus for the LDA](#prepare). This notebook demonstrates how to load a (German) TEI xml, extract metadata and texts and filter unwanted POS (only nouns are left). The result is then saved as a json which can be used in the subsequent cells. You can also prepare your corpus externally, see my [example](./scripts/text.py) which is tailored to Russian texts. It removes all non-cyrillic characters, removes all words which are not nouns and sets all nouns into first person singular using POS tagging. The result is again saved in a json file
2. [Create LDA model for the corpus](#create)
3. [Compute topic distribution for corpus](#compute)
4. [Explore corpus](corpus.ipynb) (different notebook)

Due to copyright reasons I cannot publish the scraped raw data. The results of the smoothing process in step 2 are [here](./projects/); they are used in the examples below.

In [14]:
import os
import sys
from gensim import corpora, models
import logging
import errno
import pandas as pd
from dateutil import parser
import pytz
import numpy as np
import json
import xml.etree.ElementTree as ET
import re
from tqdm import tqdm
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# set global path for corpus etc.
project = "russian_literature"
project_path = "projects" + os.path.sep + project


## Create LDA model for corpus<a id='create'></a>

This cell creates the topic model for the specified corpus stored in JSON files

[Back to top](#top)

In [15]:
number_of_topics = 100

# load corpus
corpus = []   
try:
    # load prepared corpus
    corpus = corpora.MmCorpus(os.path.join(project_path, project + ".corp"))
    dictionary = corpora.Dictionary.load(os.path.join(project_path, project + ".dict"))
except FileNotFoundError:
    json_data = open(os.path.join(project_path, project + ".json"))
    data = json.load(json_data)
    json_data.close()
    for entry in data:
        corpus.append(entry["text"])

    print("File extraction complete.")

    dictionary = corpora.Dictionary(corpus)
    dictionary.save(os.path.join(project_path, project + ".dict"))

    corpus = [dictionary.doc2bow(text) for text in corpus]
    corpora.MmCorpus.serialize(os.path.join(project_path, project + ".corp"), corpus)    

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=number_of_topics, alpha='auto', eval_every=5, passes=20)

start = 1
while os.path.isfile(os.path.join(project_path, project + "_" + +str(start)+ ".lda")):
    start += 1

lda.save(os.path.join(project_path, project + "_" +str(start)+ ".lda"))

print("LDA saved as", os.path.join(project, project + "_" +str(start)+ ".lda"))

2021-09-17 09:46:51,387 : INFO : loaded corpus index from projects/russian_literature/russian_literature.corp.index
2021-09-17 09:46:51,388 : INFO : initializing cython corpus reader from projects/russian_literature/russian_literature.corp
2021-09-17 09:46:51,392 : INFO : accepted corpus with 975 documents, 488602 features, 2354218 non-zero entries
2021-09-17 09:46:51,393 : INFO : loading Dictionary object from projects/russian_literature/russian_literature.dict
2021-09-17 09:46:51,796 : INFO : loaded projects/russian_literature/russian_literature.dict
2021-09-17 09:46:51,888 : INFO : using autotuned alpha, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01

KeyboardInterrupt: 

## Compute topic distribution for corpus<a id='compute'></a>

[Back to top](#top)

In [13]:
# load LDA model and dictionary
dictionary = corpora.Dictionary.load(os.path.join(project_path, project + ".dict"))
#model = models.LdaModel.load(os.path.join(project_path, file_name))
model = models.LdaModel.load("projects/russian_literature/russian_literature_mallet.lda")

columns = ['group', 'url', 'date', 'comment_count', 'words']
columns.extend([str(topic) for topic in range(model.num_topics)])

result = []

with open(os.path.join(project_path, project + ".json")) as json_data:
     data = json.load(json_data)

too_short = 0

for entry in data:
    # get topic distribution for entry
    line = {}
                
    # filter too short entries
    if len(entry["text"]) < 5:
        too_short += 1
        continue

    topics = [0] * model.num_topics
    for (topic, prop) in model[dictionary.doc2bow(entry["text"])]:
        topics[topic] = prop
    line["group"] = entry["volume"]+":"+entry["issue"]
    line["url"] = entry["url"]
    line["date"] = entry['date']
    line["words"] = len(entry["text"])
    try:
        line["comment_count"] = entry["comment_count"]
    except KeyError:
        line["comment_count"] = 0
    for counter in range(len(topics)):
        line[str(counter)] = topics[counter]
    result.append(line)

print("Total number of entries:", len(data))
print("Removed because too short: ", too_short)
            
frame = pd.DataFrame(result)
print(columns)
frame = frame[columns]
number = 1
if number > 0:
    file_name = project + "_topics_" + str(number) + ".json"
else: 
    file_name = project + "_topics.json"

frame.to_json(os.path.join(project_path, file_name), orient='split')
print ("Created", os.path.join(project_path, file_name))

2021-09-16 23:42:41,229 : INFO : loading Dictionary object from projects/russian_literature/russian_literature.dict
2021-09-16 23:42:41,597 : INFO : loaded projects/russian_literature/russian_literature.dict
2021-09-16 23:42:41,647 : INFO : loading LdaModel object from projects/russian_literature/russian_literature_mallet.lda
2021-09-16 23:42:41,963 : INFO : loading id2word recursively from projects/russian_literature/russian_literature_mallet.lda.id2word.* with mmap=None
2021-09-16 23:42:41,964 : INFO : loading word_topics from projects/russian_literature/russian_literature_mallet.lda.word_topics.npy with mmap=None
2021-09-16 23:42:42,101 : INFO : loading wordtopics from projects/russian_literature/russian_literature_mallet.lda.wordtopics.npy with mmap=None
2021-09-16 23:42:42,330 : INFO : loaded projects/russian_literature/russian_literature_mallet.lda
2021-09-16 23:42:42,331 : WARNING : random_state not set so using default value
2021-09-16 23:42:42,331 : INFO : dtype was not set in

2021-09-16 23:44:00,133 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-16 23:44:05,588 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-16 23:44:05,593 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-16 23:44:08,543 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num

2021-09-16 23:45:40,519 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-16 23:45:40,529 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-16 23:45:43,596 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-16 23:45:49,109 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-16 23:45:49,114 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --rem

2021-09-16 23:47:20,926 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-16 23:47:26,317 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-16 23:47:26,322 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-16 23:47:29,627 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num

2021-09-16 23:49:03,175 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-16 23:49:03,181 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-16 23:49:06,476 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-16 23:49:12,142 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-16 23:49:12,149 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --rem

...

2021-09-17 01:57:55,333 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-17 01:58:00,590 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-17 01:58:00,598 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-17 01:58:03,501 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num

2021-09-17 01:59:31,769 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-17 01:59:31,779 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-17 01:59:34,827 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-17 01:59:40,264 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-17 01:59:40,269 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --rem

2021-09-17 02:01:05,997 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-17 02:01:11,217 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-17 02:01:11,222 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-17 02:01:14,150 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num

2021-09-17 02:02:41,912 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-17 02:02:41,914 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/b011e4_corpus.txt --output /tmp/b011e4_corpus.mallet.infer --use-pipe-from /tmp/b011e4_corpus.mallet
2021-09-17 02:02:44,819 : INFO : inferring topics with MALLET LDA '/home/ghowa/working/mallet-2.0.8/bin/mallet infer-topics --input /tmp/b011e4_corpus.mallet.infer --inferencer /tmp/b011e4_inferencer.mallet --output-doc-topics /tmp/b011e4_doctopics.txt.infer --num-iterations 100 --doc-topics-threshold 0.0 --random-seed 0'
2021-09-17 02:02:50,068 : INFO : serializing temporary corpus to /tmp/b011e4_corpus.txt
2021-09-17 02:02:50,074 : INFO : converting temporary corpus to MALLET format with /home/ghowa/working/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --rem

Total number of entries: 975
Removed because too short:  0
['group', 'url', 'date', 'comment_count', 'words', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
Created projects/russian_literature/russian_literature_topics.json
